In [1]:
# Code to get all the revised LC points (from rerunning EITfluxsum.py for subsets of the years 1996-2022) into the same results file
# and all the metadata files into the same metadata file

# also revise the EIT pinhole flag to only apply to images which have filter = 'Clear'

In [1]:
import EITfunc.EITlook as eit

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clrs
import matplotlib.cm as cmx
import matplotlib.colorbar as cb
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import gridspec

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

import os

from astropy import units as u
from astropy.constants import G
from astropy.io import ascii, fits
from astropy.table import Table, vstack
from astropy.timeseries import LombScargle
import astropy.time
from astropy.coordinates import HeliocentricMeanEcliptic, SkyCoord

import sunpy.coordinates

import scipy.optimize as sciop
from scipy.stats import mode, binned_statistic

import time
import copy

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
ls ./EIT/EITresults

EIT_LC_1996_1997.txt    EIT_LC_2004_2025.txt    EITdark.py*
EIT_LC_1997_2001.txt    EIT_LC_withflags.txt    EITfluxsum.py*
EIT_LC_2001_2004.txt    EIT_headerMetadata.txt  EITmetadata.py


In [3]:
start_years = np.array((1996,1997,2001,2004))
end_years = np.array((1997,2001,2004,2025))

In [4]:
LCheader = 't_obs[JD] solar_distance[m] max_annular_pixel_radius[pix] wavelength[angstrom] unnorm_flux[DN/s] unnorm_flux_unc[DN/s] pinholes[1ifYes] cameraError[1ifYes]'
metadataHeader = 'DATE-BEG DATE-AVG XPOSURE CMDXPOS SHUTCLS FILTER WAVELNTH OBJECT SCI_OBJ OBS_PROG CMP_NO UCD EXPMODE LYNESYNC ROUTPORT NLEBPROC LEBPROC1 LEBPROC2 LEBPROC3 WCSNAME CTYPE1 CTYPE2 CUNIT1 CUNIT2 PC1_1 PC1_2 PC2_1 PC2_2 CDELT1 CDELT2 CRVAL1 CRVAL2 CRPIX1 CRPIX2 CROTA CFTEMP CCDTEMP SOLAR_B0 RSUN_ARC RSUN_OBS RSUN_REF CAR_ROT DSUN_OBS SC_X0 SC_Y0 SC_ROLL HAEX_OBS HAEY_OBS HAEZ_OBS'

In [5]:
allLC = np.zeros((1,8))
#allmetadata = Table.read("./EIT/EITresults/EIT_headerMetadata_1996_1999.txt",format='ascii')
#print(len(allmetadata))

for i in range(len(start_years)):
    start_yr = start_years[i]
    end_yr = end_years[i]

    LCdata = np.genfromtxt("./EIT/EITresults/EIT_LC_{0}_{1}.txt".format(start_yr,end_yr))
    print(np.shape(LCdata))
    allLC = np.vstack((allLC, LCdata))
    '''
    if i > 0:
        metadata = Table.read("./EIT/EITresults/EIT_headerMetadata_{0}_{1}.txt".format(start_yr,end_yr),format='ascii')
        print(len(metadata))
        metadata['CMP_NO'] = metadata['CMP_NO'].astype(str)
        allmetadata = vstack((allmetadata,metadata))
    '''
allLC = allLC[1:]
print(np.shape(allLC))
#print(len(allmetadata))

# sort by observation time
t_idxs = np.argsort(allLC[:,0])
allLC = allLC[t_idxs]

#t_idxs_metadata = np.argsort(allmetadata['DATE-BEG'])
#allmetadata = allmetadata[t_idxs_metadata]
#print(allmetadata)

(2877, 8)
(16142, 8)
(14496, 8)
(61285, 8)
(94800, 8)


In [6]:
allmetadata = Table.read("./EIT/EITresults/EIT_headerMetadata.txt",format='ascii')

In [7]:
print(np.allclose(allLC[:,0], allmetadata['DATE-BEG'], 1.e-12))
print(allLC[:,0])
print(allmetadata['DATE-BEG'])

True
[2450098.509462 2450098.520198 2450098.53664  ... 2460412.545185
 2460412.550544 2460412.554643]
     DATE-BEG     
------------------
 2450098.509462373
2450098.5201983913
2450098.5366403703
  2450099.07985044
2450102.2426729975
 2450102.278504838
 2450102.401938831
2450103.1874925694
 2450103.212478449
2450103.2389932405
               ...
 2460411.050899074
2460411.5414336575
2460412.0411972683
2460412.0453280094
2460412.0506855557
2460412.0547792595
 2460412.541053646
2460412.5451847916
  2460412.55054353
2460412.5546430093
Length = 94800 rows


In [8]:
print(np.unique(allmetadata['FILTER']))

#print(len(allmetadata[allmetadata['FILTER'] == 'Al_+1']))
#print(len(allmetadata[allmetadata['FILTER'] == 'Al_sup']))
#print(len(allmetadata[allmetadata['FILTER'] == 'Clear']))


# number of pinholes
#print(len(allLC[allLC[:,6] == 0]))
print(len(allLC[allLC[:,6] == 1]))

Al_filterMask = (allmetadata['FILTER'] != 'Clear')
#print(len(allmetadata[~Al_filterMask]))


#get rid of spurious detections of pinholes where there can't have been any because the filter was in place
allLC[:,6][Al_filterMask] = 0

print(len(allLC[allLC[:,6] == 1]))


FILTER
------
 Al_+1
Al_sup
 Clear
10788
1495


In [9]:
#pinholes = allLC[:,5]

Al_filterMask = (allmetadata['FILTER'] == 'Al_+1') | (allmetadata['FILTER'] == 'Al_sup')
print(len(allmetadata[~Al_filterMask]))

#get rid of spurious detections of pinholes where there can't have been any because the filter was in place
allLC[:,6][Al_filterMask] = 0

print(len(allLC[allLC[:,6] == 1]))


6083
1495


In [10]:
np.savetxt("./EIT/EITresults/EIT_LC_withflags.txt", allLC, fmt="%f", header=LCheader)
ascii.write(allmetadata, "./EIT/EITresults/EIT_headerMetadata.txt",overwrite=True)